# Half-Automatic Pitch Track Smoother

This notebook is for **reading a manually edited** pitch track and **automatically**,
    
    1)Octave Correcting,
    
    2)Smoothing the edited regions,
    
    3)Zeroeing frequencies below a threshold and,
    
    4)Unifying the frame rate.
    
**Guide:** If your task includes octave corrections start from section A and follow the guide, else start from section B, part 1.2

In [20]:
import os
import numpy as np

# Specify the frequency threshold
threshold = 50.0

## A) Octave Correction

This part is for correcting the regions where there is an octave error with specifying the time instants and the factor of error.

You can select an **unedited pitch track** with the cell below or,

In [5]:
ID = "a80b0276-f769-433c-8944-d316848409c5"
dataDir0 = os.path.join('..','data','pitchTracks','unedited_pitchTracks')

pitchFile = os.path.join(dataDir0,'{}.pitch'.format(ID))

newFileName = ID+".txt"

select an **edited pitch track** with the cell below here.

In [6]:
recordName = 'Yüzüdür_Cihanı'

try_no = 1 # for keeping track of the edition

fileName = recordName+'_edited{}.txt'.format(try_no)
dataDir0 = os.path.join('..', 'metaData', 'edited_pitchTracks',fileName) # input directory
pitchFile = dataDir0

newFileName = recordName+'_autoSmooth{}.txt'.format(try_no)
dataDir1 = os.path.join('..', 'metaData', 'smooth_pitchTracks',newFileName) # output directory

You can read the **pitch track file** with the cell below. However, if you want to read a file with **more than 2 columns** you need to **delete** the **third** column.

In [19]:
time = []
freq = []

with open(pitchFile,"r") as file1:
    
    for line in file1:
        
        lst = line.split('\t')
        
        t = lst[0]
        f = float(lst[1])
        
        if f < threshold:
            f = 0.0
                
        time.append(t)
        freq.append(f)
        
digits = len(time[10].split('.')[1])

if digits < 2: 
    digits = 2

print('Please use {} digits after the decimal for time specification.'.format(digits))

Please use 9 digits after the decimal for time specification.


Specify the regions with octave errors in **sec.msec** in a **string**. 

Follow the **printout of the above cell** for time specification.

**Note:** Remember to put 0.01 sec after the end of the interval (inclusion) and numbers that are integer multiples of 0.1 should be written up to one point after the decimal.

In [14]:
start_times = ["132.650000000"]
end_times =   ["136.500000000"]

factors = [1/2]

In [15]:
start_pos = []
end_pos = []

In [16]:
for s in start_times:
    start_pos.append(time.index(s))
        
for e in end_times:
    end_pos.append(time.index(e))

In [17]:
if len(start_pos) == len(end_pos):
    
    print('Regions are clear, octaves corrected')
    
    for k in range(len(start_pos)):

        interval = range(start_pos[k],end_pos[k])

        for i in interval: 
            freq[i] = freq[i]*factors[k]
else:
    print("Regions Unclear!Couldn't clear")

Regions are clear, octaves corrected


You can **create the octave corrected text file** directly using the cell below **or continue with smoothing** in part **B)**.

In [ ]:
with open(newFileName,"w") as newFile:
    
    for i in range(len(time)):
        
        t  = time[i]
        f = str(freq[i])
        
        newFile.write(t+'\t'+f+'\n')

## B) Edited Pitch Track Smoothing

This section is for:

    1) Automatically finding the parts that you have edited the pitch track manually and smoothing them,

    2) Zeroing out certain frequencies below a threshold.

    3) Unifying the frame rate.

![gap%20filling.PNG](attachment:gap%20filling.PNG)

### 1.1) Octave Corrected Files

If you **made octave corrections** in the part A, **finish this part** and **move** to the part **2)**,

**otherwise**, go to part **1.2)**

In [9]:
points = [] 

In [10]:
for t in time:
    
    # get the new/old point information    
    if (t.split(".")[1][2:] == "0000000") :
        status = "Old Point"    
    else:
        status = "New Point"

    points.append(status) 

points = np.array(points)   

### 1.2) Retrieve the Info.

In [21]:
recordName = 'egin_havası'

try_no = 1 # for keeping track of the edition

fileName = recordName+'_edited{}.txt'.format(try_no)
dataDir0 = os.path.join('..', 'metaData', 'edited_pitchTracks',fileName) # input directory

newFileName = recordName+'_autoSmooth{}.txt'.format(try_no)
dataDir1 = os.path.join('..', 'metaData', 'smooth_pitchTracks',newFileName) # output directory

In [22]:
with open(dataDir0) as file:
    txt =file.readlines()

In [23]:
time = []
freq = []
points = [] 

In [24]:
for line in txt:
    
    # get the time and frequency
    lst = line.split('\t')
    t = lst[0]
    f = float(lst[1])
    
    time.append(t)
    freq.append(f)

    # get the new/old point information    
    if (t.split(".")[1][2:] == "0000000") :
        status = "Old Point"    
    else:
        status = "New Point"

    points.append(status) 

points = np.array(points)

### 2) Find where the New Points are

In [25]:
indices = np.where(points == 'New Point')[0]
delta = np.diff(indices) # distances between the new points

In [26]:
# Specify a distance limit for seperation
delta_max = 15 

### 3) Find the boundaries of the intervals

In [13]:
boundaries = [0]  

for counter, value in enumerate(delta):
    
    if value > delta_max:
        boundaries.append(counter)
        boundaries.append(counter+1)

boundaries.append(len(delta))

intervals = indices[boundaries]

### 4) Smooth the intervals

In [14]:
for n in range(int(len(intervals)/2)):
    
    start = intervals[2*n]
    end = intervals[2*n+1]
    
    for index in range(start,end):
        
        t = time[index]        
        if (t.split(".")[1][2:] == "0000000"):    
            
            # 2p-smoothing
            freq[index] = (freq[index-1]+freq[index+1])/2

### 5) Write to Text

In [27]:
with open(dataDir1,"w") as newFile:

    for index in range(len(time)):
        
        t = time[index]
        f = str(freq[index])
        
        # zero out  frequencies
        if freq[index] < threshold:
            f = '0.0'
        
        # unify the frame rate
        if t.split(".")[1][2:] == "0000000":            
            newFile.write(t+'\t'+f+'\n')